# Load Model

In [1]:
import sys
sys.path.append("../..")

import os

from lightning_modules import FeatureLSTMModule
from utils.io import load_yaml


server = "local"
servers = load_yaml("../../config/servers.yml")
server = servers[server]

log_location = "miccai/feature_lstm/phantom/resnet34/frame_increment_1/transforms/version_0"
checkpoint = "checkpoints/epoch=15-step=320.ckpt"

config = load_yaml(os.path.join(server["logging"]["location"], log_location, "config.yml"))
model = FeatureLSTMModule.load_from_checkpoint(os.path.join(server["logging"]["location"], log_location, checkpoint), **config["model"])

/home/martin/miniconda3/envs/torch110/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Reading file from ../../config/servers.yml
Reading file from /media/martin/Samsung_T5/logs/miccai/feature_lstm/phantom/resnet34/frame_increment_1/transforms/version_0/config.yml


# Load Data

In [6]:
import cv2
import numpy as np
import pandas as pd
from utils.processing import differentiate_duv

prefix = "21_11_25_first_test_data_frames"
file = "pre_processed_10th_frame_log_test_train_no_nan.pkl"
df = pd.read_pickle(
    os.path.join(server["database"]["location"], prefix, file)
)
df = df[df.train == False]

from datasets import ImageSequenceDuvDataset
ds = ImageSequenceDuvDataset(
    df,
    os.path.join(server["database"]["location"], prefix),
    seq_len=20,
    frame_increment=1,
    frames_between_clips=20
)

from kornia import tensor_to_image
from utils.viz import create_blend_from_four_point_homography
from utils.io import generate_path


for imgs, duvs, frame_idx, vid_idx in ds:
    output_path = f"/tmp/blends/vid_{vid_idx}"
    generate_path(output_path)
    imgs = imgs.unsqueeze(0).to(model.device).float()/255.
    duvs = duvs.unsqueeze(0).to(model.device).float()
    
    # derivative
    dduvs = differentiate_duv(duvs)

    duvs_ip1 = model(imgs[:, 2:], duvs[:, 1:-1], dduvs[:, :-1])
    
    batch = 0
    blends = create_blend_from_four_point_homography(imgs[batch, 2:-1], imgs[batch, 3:], duvs_ip1[batch,:-1])

    for idx, blend in enumerate(blends):
        blend = tensor_to_image(blend, False)
        blend = (blend*255.).astype(np.uint8)
        blend = cv2.resize(blend, [640, 480])
        
        cv2.imwrite(f"{output_path}/blend_{idx}.png", blend)
        
    

Generating path at /tmp/blends/vid_1
Generating path at /tmp/blends/vid_8
